# Data Analysis

In [44]:
import os
import glob
import shutil
import numpy as np
from edd.data import IBMQData
import edd.workflow as wf

cwd = os.getcwd()
device = "Bogota"

base_dir = f"{cwd}/{device}/HaarData"

## Step 1: Load in Raw Data and Parse It

In [45]:
result_dir = f"{base_dir}/rawHaarData"
files = glob.glob(f"{result_dir}/*.yml")

In [46]:
def compute_fid(counts):
    """
    Computes fidelity as num0 / (num0 + num1).
    """
    num0 = counts.get('0x0', 0)
    num1 = counts.get('0x1', 0)
    
    return num0 / (num0 + num1)

def compute_error(counts, resamples=1000):
    """
    Computes 2 \sigma error on fidelity.
    """
    num0 = counts.get('0x0', 0)
    num1 = counts.get('0x1', 0)
    tot_counts = num0 + num1
    prob0 = num0 / tot_counts
    
    fids = []
    for j in range(resamples):
        re_counts = np.random.choice([0, 1], size=tot_counts, p=[prob0, 1 - prob0]) 
        fid = 1 - sum(re_counts) / tot_counts
        fids.append(fid)
        
    return 2 * np.std(fids)

def boot_fids(fids, resamples=1000):
    """
    Computes 2 \sigma fidelity bootstrap.
    """
    means = []
    for r in range(resamples):
        sample = np.random.choice(fids, len(fids))
        avg_fid = np.mean(sample)
        means.append(avg_fid)
        
    return 2 * np.std(means)

def compute_error(counts, resamples=1000):
    """
    Faster way compute errors.
    """
    return 0.0033

## Step 1: Summarize the data

In [47]:
min_delay_error = []
min_delay_dict = {}
opt_delay_error = []
opt_delay_dict = {}

for f in files:
    # load and format data
    data = IBMQData()
    data.load_raw_data(f)
    data.format_data()
    # extract sequence and counts
    t_data = data.tuple_data
    job_info = t_data[0][0]
    s_info = job_info.split("_")
    dtype_idx = s_info.index("dtype") + 1
    d_type = s_info[dtype_idx]
    seq_idx = s_info.index("decay") + 1
    seq = s_info[seq_idx]
    if seq == "uddx":
        seq += s_info[seq_idx + 1]
    elif seq == "qdd":
        seq += f"-{s_info[seq_idx + 1]}-{s_info[seq_idx + 2]}"
    elif seq == "super":
        seq += f"-{s_info[seq_idx + 1]}"
    counts_list = [t_data[j][2] for j in range(len(t_data))]
    fid_list = [compute_fid(count) for count in counts_list]
    point_err = compute_error(counts_list[0])
    
    # add data to appropriate dictionary
    if d_type == "min":
        min_delay_error.append(point_err)
        if seq in min_delay_dict:
            min_delay_dict[seq].extend(fid_list)
        else:
            min_delay_dict[seq] = fid_list
    else:
        opt_delay_error.append(point_err)
        if seq in opt_delay_dict:
            opt_delay_dict[seq].extend(fid_list)
        else:
            opt_delay_dict[seq] = fid_list

In [48]:
for key in opt_delay_dict:
    print(f"opt delay: {key}", np.mean(opt_delay_dict[key]))
    print(f"min delay: {key}", np.mean(min_delay_dict[key]))

opt delay: purex 0.9251558999999999
min delay: purex 0.8599283333333333
opt delay: xy4 0.9163034999999999
min delay: xy4 0.8689506666666666
opt delay: qdd-1-4 0.9072668749999999
min delay: qdd-1-4 0.9355783333333333
opt delay: rga32c 0.919916375
min delay: rga32c 0.9383401666666664
opt delay: ur20 0.9361291249999999
min delay: ur20 0.9496033333333332
opt delay: rga64c 0.9195898749999999
min delay: rga64c 0.9280875
opt delay: qdd-1-2 0.9174185
min delay: qdd-1-2 0.9404848333333333
opt delay: cdd3 0.9293168749999999
min delay: cdd3 0.9459957499999999
opt delay: super-hahn 0.9190887
min delay: super-hahn 0.9207173333333333


## Step 2: Make the CSV File Summary

In [49]:
min_fname = f"{base_dir}/csvHaarData/minDelay_haarData_{device}.csv"
with open(min_fname, "w") as f:
    for seq in min_delay_dict:
        seq = seq.replace("_", "-")
        line = f"{seq},{','.join(str(x) for x in min_delay_dict[seq])}\n"
        f.write(line)
        
opt_fname = f"{base_dir}/csvHaarData/optDelay_haarData_{device}.csv"
with open(opt_fname, "w") as f:
    for seq in opt_delay_dict:
        seq = seq.replace("_", "-")
        line = f"{seq},{','.join(str(x) for x in opt_delay_dict[seq])}\n"
        f.write(line)

## Step 5: Turn _ into - for sequence names in csv handles

In [50]:
files = glob.glob(f"{base_dir}/csvDelayData/*.csv")
import os

In [51]:
for f in files:
    split_f = f.split("_")
    seq_idx = split_f.index("seq") + 1
    seq = split_f[seq_idx]
    if seq == "qdd":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}-{split_f[seq_idx + 2]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f[seq_idx + 2] = None
        split_f.remove(None)
        split_f.remove(None)
        
    if seq == "super":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f.remove(None)
    new_f = "_".join(split_f)
    
    os.rename(f, new_f)

In [14]:
data_dir = "/Users/worknic/Documents/Armonk_Bogota_24AugData/Bogota/q3/min_delay/"

for file in data_files:
    file = file + ".csv"
    shutil.copy(file, data_dir)